#Installing all the required library, run this first.


In [ ]:
pip install fuzzywuzzy

In [ ]:
pip install country_converter


In [ ]:
#Function that runs the entire match process. Run the above code first.

def run_match(x,y) :
  
  #import all the required python libraries
  import pandas as pd
  import numpy as np

  
  #import the  two excel files
  df_1 = pd.read_excel(x)
  df_2 = pd.read_excel(y)

  # Converting cuntry code to country name 
  import country_converter as coco

  # add country name by applying the convert method
  df_2['Country'] = df_2.OWNR_COUNTRY.apply(lambda x: coco.convert(names=x, to='name_short', not_found=None))

  states = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI"}

  #if you want to invert the dictionary
  # abbrev_to_us_state = dict(map(reversed, us_state_to_abbrev.items()))

  df_1['State'] = df_1['State'].str.strip().replace(states)

  #converting fileds to string type

  #Table 1
  df_1['Street']= df_1['Street'].astype(str)
  df_1['City'] = df_1['City'].astype(str)
  df_1['State'] = df_1['State'].astype(str)
  df_1['Postal Code'] = df_1['Postal Code'].astype(str)
  df_1['Country/Region'] = df_1['Country/Region'].astype(str)
  df_1['Account'] = df_1['Account'].astype(str)

  #Table 2
  df_2['OWNR_ADDRESS_1']=df_2['OWNR_ADDRESS_1'].astype(str)
  df_2['OWNR_CITY']=df_2['OWNR_CITY'].astype(str)
  df_2['OWNR_STATE']=df_2['OWNR_STATE'].astype(str)
  df_2['OWNR_ZIPCODE']=df_2['OWNR_ZIPCODE'].astype(str)
  df_2['Country']=df_2['Country'].astype(str)
  df_2['SRC_FIRST_NM']=df_2['SRC_FIRST_NM'].astype(str)
  df_2['SRC_SURNM']=df_2['SRC_SURNM'].astype(str)
  df_2['Company Name']=df_2['Company Name'].astype(str)
  

  # Concatenating the different  address columns to one 

  # to add account names simply add this below - df_1['Account'] + ", " +

  df_1['fulladdress'] =  df_1['Account'] + ", " + df_1['Street'] + ", " + df_1['City'] + ", " + df_1['State'] + ", " + df_1['Postal Code'] + ", " + df_1['Country/Region']

  # Concatinating the differnt address columns to one 

  #df_2['SRC_FIRST_NM'] + ", " + df_2['SRC_SURNM'] + ", " +

  df_2['fulladdress'] =  df_2['Company Name'] + ", " + df_2['OWNR_ADDRESS_1'] + ", " + df_2['OWNR_CITY'] + ", " + df_2['OWNR_STATE'] + ", " + df_2['OWNR_ZIPCODE'] + ", " + df_2['Country']
  
  #Converting full address into lower case
  
  df_1['fulladdress'] = df_1['fulladdress'].str.lower()
  df_2['fulladdress'] = df_2['fulladdress'].str.lower()
    
  df1 = df_1
  df2 = df_2


  import fuzzywuzzy
  from fuzzywuzzy import fuzz
  from fuzzywuzzy import process

  similarity = []
  address_table1 = []
  #to sure sort method and , scorer=fuzz.ratio

  for i in df2.fulladdress:
          ratio = process.extract( i, df1.fulladdress, limit=1, scorer = fuzz.ratio)
          address_table1.append(ratio[0][0])
          similarity.append(ratio[0][1])

  df2['address_table1'] = pd.Series(address_table1)
  df2['similarity'] = pd.Series(similarity)

  #Saving different segments to separate excel sheets

  #100% Match
  df_match100 = df2[df2['similarity'] == 100]

  #match with score more than80%
    
  df_match80= df2[df2['similarity'] > 80]

  #match with score between 50% and 80%
  df_match50_80 = df2[(df2['similarity'] > 50) & (df2['similarity']< 80) ]

  # match with score less than 50%
  df_match50 = df2[df2['similarity'] < 50]

  # Lets export all the above matches to separate excel sheets

  # Specify the name of the excel file
  file_name1 = '100%_match.xlsx'
  file_name2 = 'above80%_match.xlsx'
  file_name3 = 'between80and50%_match.xlsx'
  file_name4 = 'below50%_match.xlsx'

  # saving the excelsheet
  df_match100.to_excel(file_name1)
  df_match80.to_excel(file_name2)
  df_match50_80.to_excel(file_name3)
  df_match50.to_excel(file_name4)


In [ ]:
#df_1 = pd.read_excel('Lead Report_June 2022.xlsx')
#df_2 = pd.read_excel('Q_OWNERS-May 2022.xlsx')
run_match('Lead Report_June 2022.xlsx', 'Q_OWNERS-May 2022.xlsx')